## Segmenting and Clustering Neighborhoods in Toronto

In [1]:
# Importing all dependencies we'll need
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
import bs4 # BeautifulSoup Package
from bs4 import BeautifulSoup

#### From the Wikipedia Page - Toroto is made up of 140 Neighbours - We are going to create a dataset with all the 140 Neighbours alongside their Municipality Codes

In [3]:
wikipedia_page = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
results = requests.get(wikipedia_page)

In [4]:
wikipedia_html = results.text

In [6]:
soup = BeautifulSoup(wikipedia_html, 'html.parser')
for table in soup.find_all('table'):
    print(table.get('class'))

['wikitable', 'sortable']
['multicol']
None
['navbox']
None


In [10]:
neighborhood_html_table = soup.find_all('table')[0]
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)
df.head

AttributeError: 'list' object has no attribute 'head'

In [24]:
# Dictionary with District Number and Neighbours Included
data_dict = {"district_number":[], "neighbour":[]}

# Get the class with wikitable
neighborhood_html_table = soup.find_all('table')[6]
# neighborhood_html_table.find_all("tr")[1:]

# for row in neighborhood_html_table.find_all("tr")[1:]:
# if run again delete values of data_dict
for row in neighborhood_html_table.find_all("tr")[1:]:
    for td in row.find_all("td"):
        if len(td.findChildren()) == 0:
            district_number = td.get_text()
        else:
            neighbours = td.get_text().split(',')
            
    data_dict["district_number"].append(district_number)
    data_dict["neighbour"].append(neighbours)
        
data_dict        

{'district_number': ['C01\n',
  'C02\n',
  'C03\n',
  'C04\n',
  'C06\n',
  'C07\n',
  'C08\n',
  'C09\n',
  'C10\n',
  'C11\n',
  'C13\n',
  'C14\n',
  'C15\n',
  'E01\n',
  'E02\n',
  'E03\n',
  'E04\n',
  'E05\n',
  'E06\n',
  'E08\n',
  'E09\n',
  'E10\n',
  'E11\n',
  'W01\n',
  'W02\n',
  'W03\n',
  'W04\n',
  'W05\n',
  'W06\n',
  'W07\n',
  'W08\n',
  'W09\n',
  'W10\n'],
 'neighbour': [['Downtown',
   ' Harbourfront',
   ' Little Italy',
   ' Little Portugal\n'],
  ['The Annex',
   ' Yorkville',
   ' South Hill',
   ' Summerhill',
   ' Wychwood Park',
   ' Deer Park\n'],
  ['Forest Hill South',
   ' Oakwood–Vaughan',
   ' Humewood–Cedarvale',
   ' Corso Italia\n'],
  ['Bedford Park',
   ' Lawrence Manor',
   ' North Toronto',
   ' Forest Hill North\n'],
  ['North York', ' Clanton Park', ' Bathurst Manor\n'],
  ['Willowdale',
   ' Newtonbrook West',
   ' Westminster–Branson',
   ' Lansing-Westgate\n'],
  ['Cabbagetown',
   ' St. Lawrence Market',
   ' Toronto waterfront',
   ' 

In [88]:
# DataFrame to Organize our Dataset
column_names = ["District", "Neighbour", "Latitude", "Longitude"]
toronto = pd.DataFrame(columns=column_names)
toronto

,District,Neighbour,Latitude,Longitude


In [89]:
for i in range(len(data_dict["district_number"])):
    district_number = data_dict["district_number"][i]
    for neighbour in range(len(data_dict["neighbour"][i])):
        toronto = toronto.append({
            'District': district_number,
            'Neighbour': data_dict["neighbour"][i][neighbour]
        }, ignore_index=True)
        
toronto

,District,Neighbour,Latitude,Longitude
0,C01\n,Downtown,NaN,NaN
1,C01\n,Harbourfront,NaN,NaN
2,C01\n,Little Italy,NaN,NaN
3,C01\n,Little Portugal\n,NaN,NaN
4,C02\n,The Annex,NaN,NaN
5,C02\n,Yorkville,NaN,NaN
6,C02\n,South Hill,NaN,NaN
7,C02\n,Summerhill,NaN,NaN
8,C02\n,Wychwood Park,NaN,NaN
9,C02\n,Deer Park\n,NaN,NaN


In [90]:
### Remove the /n from the values
"C01\n".replace("\n","")

toronto.District = toronto.District.apply(lambda s: s.strip().replace("\n",""))
toronto.Neighbour = toronto.Neighbour.apply(lambda s: s.strip().replace("\n",""))
toronto

,District,Neighbour,Latitude,Longitude
0,C01,Downtown,NaN,NaN
1,C01,Harbourfront,NaN,NaN
2,C01,Little Italy,NaN,NaN
3,C01,Little Portugal,NaN,NaN
4,C02,The Annex,NaN,NaN
5,C02,Yorkville,NaN,NaN
6,C02,South Hill,NaN,NaN
7,C02,Summerhill,NaN,NaN
8,C02,Wychwood Park,NaN,NaN
9,C02,Deer Park,NaN,NaN


In [96]:
### Get Latitude and Longitude through Geocoding - test for one row
address = "Playter" + ", Toronto"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Playter, Toronto are 43.6761856, -79.3606405288553.


In [92]:
neighbourhoods_array = toronto["Neighbour"].values

In [93]:
columns = ["Neighbourhood", "Latitude", "Longitude"]
neighbours_with_lat_long = pd.DataFrame(columns=columns)
neighbours_with_lat_long

,Neighbourhood,Latitude,Longitude


In [94]:
for neighbourhood in neighbourhoods_array:
    try:
        address = neighbourhood + ", Toronto"
        geolocator = Nominatim(user_agent="toronto_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
    except:
        latitude = None
        longitude = None
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Downtown, Toronto are 43.6541737, -79.3808116451341.
The geograpical coordinate of Harbourfront, Toronto are 43.6400801, -79.3801495.
The geograpical coordinate of Little Italy, Toronto are 43.6552079, -79.4148769.
The geograpical coordinate of Little Portugal, Toronto are 43.6496309, -79.4314332.
The geograpical coordinate of The Annex, Toronto are 43.6703377, -79.407117.
The geograpical coordinate of Yorkville, Toronto are 43.6713861, -79.3901677.
The geograpical coordinate of South Hill, Toronto are 43.6779261, -79.4057671.
The geograpical coordinate of Summerhill, Toronto are 43.6816776, -79.3905037.
The geograpical coordinate of Wychwood Park, Toronto are 43.677743, -79.4201849779446.
The geograpical coordinate of Deer Park, Toronto are 43.68809, -79.3940935.
The geograpical coordinate of Forest Hill South, Toronto are 43.6935586, -79.4139023.
The geograpical coordinate of Oakwood–Vaughan, Toronto are 43.6916811, -79.4399097.
The geograpical coordinat

### Correct the Neighborhood Names without Latitude

In [125]:
# Missing latitude and longitude 
    # Humewood-Cedarvale  --- will use Humewood
    # Westminster-Branson         --- will use Westminster
    # Parkwoods-Donalda --- will use Parkwoods
    # Danforth (Greektown) --- will use Danforth
    # Playter Estates ---- will use Playter Estates
    # Eglinton East (Knob Hill) --- will use Eglinton East
    # Rockcliffe–Smythe ---- will use Rockcliffe
    # Weston-Pellam Park ---- will use weston pellam park
    # Amesbury (Brookhaven) ---- will use Amesbury
    # Fairbank (Briar Hill-Belgravia) ---- will use Fairbank
    # Humbermede (Emery) ---- will use Humbermede
    # Jane and Finch (Black Creek or Glenfield-Jane Heights) ---- will use Black Creek
    # Sunnylea (The Queensway – Humber Bay)------ will use Sunnylea
    # Eringate – Centennial – West Deane ----- will use Eringate
    # Princess-Rosethorn ---- will use Rosethorn
    # Kingsview Village-The Westway --- will use Kingsview Village
    # Richview (Willowridge) --- Willowridge
    # Clairville ----- Claireville, Brampton
    # Thistletown - Beaumond Heights --- Thistletown
    # Smithfield: Mount Olive-Silverstone-Jamestown ---- will be Smithfield

# replace_list = [
#     ["Humewood–Cedarvale", "Humewood"],
#     ["Westminster–Branson", "Westminster"],
#     ["Parkwoods–Donalda","Parkwoods"],
#     ["Danforth (Greektown)", "Danforth"],
#     ["Playter Estates", "Playter"],
#     ["Eglinton East (Knob Hill)", "Elglinton East"],
#     ["Rockcliffe–Smythe", "Rockcliffe"],
#     ["Weston-Pellam Park", "weston pellam park"],
#     ["Amesbury (Brookhaven)", "Amesbury"],
#     ["Fairbank (Briar Hill-Belgravia)", "Fairbank"],
#     ["Humbermede (Emery)", "Hubermede"],
#     ["Jane and Finch (Black Creek or Glenfield-Jane Heights)", "Black Creek"],
#     ["Sunnylea (The Queensway – Humber Bay)", "Sunnylea"],
#     ["Eringate – Centennial – West Deane", "Eringate"],
#     ["Princess-Rosethorn", "Rosethorn"],
#     ["Kingsview Village-The Westway", "Kingsview Village"],
#     ["Richview (Willowridge)", "Willowridge"],
#     ["Clairville", "Claireville, Brampton"],
#     ["Thistletown - Beaumond Heights", "Thistletown"],
#     ["Smithfield: Mount Olive-Silverstone-Jamestown", "Smithfield"]
# ]

replace_list = [
    ["Woodbine Corridor", "Woodbine - Lumsden"],
    ["Elglinton East", "Eglinton East"],
    ["Bendale (Cedarbrae)","Bendale"],
    ["weston pellam park", "Weston Pellam"],
    ["Hubermede", "Humbermede"]
]

for i in range(len(replace_list)):
     toronto.Neighbour[toronto.Neighbour.values == replace_list[i][0]] = replace_list[i][1]

toronto.head()

,District,Neighbour,Latitude,Longitude
0,C01,Downtown,NaN,NaN
1,C01,Harbourfront,NaN,NaN
2,C01,Little Italy,NaN,NaN
3,C01,Little Portugal,NaN,NaN
4,C02,The Annex,NaN,NaN


In [131]:
toronto.loc[0, "Neighbour"]

'Downtown'

### Regenerate Latitude and Logitudes and append to DataFrame

In [133]:
neighbourhoods_array = toronto.Neighbour.values
for i in range(len(toronto.Neighbour)):
    try:
        address = toronto.loc[i, "Neighbour"] + ", Toronto"
        geolocator = Nominatim(user_agent="toronto_explorer")
        location = geolocator.geocode(address)
        toronto.loc[i, "Latitude"] = location.latitude
        toronto.loc[i, "Longitude"] = location.longitude
#         print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
    except:
        toronto.loc[i, "Latitude"] = np.nan
        toronto.loc[i, "Longitude"] = np.nan
#         print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

toronto.head()

,District,Neighbour,Latitude,Longitude
0,C01,Downtown,43.6542,-79.3808
1,C01,Harbourfront,43.6401,-79.3801
2,C01,Little Italy,43.6552,-79.4149
3,C01,Little Portugal,43.6496,-79.4314
4,C02,The Annex,43.6703,-79.4071


In [141]:
toronto[toronto.Latitude.isna()]

,District,Neighbour,Latitude,Longitude
53,E02,Woodbine - Lumsden,NaN,NaN
99,W03,Weston Pellam,NaN,NaN
111,W05,Humbermede,NaN,NaN


In [144]:
# Inserting lat and longitude for the missing columns from the internet
toronto.loc[53,"Latitude"], toronto.loc[53,"Longitude"] = ("43.671780", "-79.322890")
toronto.loc[99,"Latitude"], toronto.loc[99,"Longitude"] = ("43.663380", "-79.438250")
toronto.loc[111,"Latitude"], toronto.loc[111,"Longitude"] = ("43.695830", "-79.536530")
toronto.isna().sum()

District     0
Neighbour    0
Latitude     0
Longitude    0
dtype: int64

In [151]:
toronto.dtypes

District     object
Neighbour    object
Latitude     object
Longitude    object
dtype: object

In [153]:
# Convert Latitude and Longitude to Float
toronto.Latitude = toronto.Latitude.astype('float64')
toronto.Longitude = toronto.Longitude.astype('float64')
toronto.dtypes

District      object
Neighbour     object
Latitude     float64
Longitude    float64
dtype: object

#### Examining the Dataframe

In [154]:
print('The dataframe has {} districts and {} neighborhoods.'.format(
        len(toronto['District'].unique()),
        toronto.shape[0]
    )
)

The dataframe has 33 districts and 134 neighborhoods.


In [155]:
# Get the Latitude and Longitude of Toronto
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_geo")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


### Map of Toronto with Neighbours Superimposed

In [156]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district, neighbor in zip(toronto['Latitude'], toronto['Longitude'], toronto['District'], toronto['Neighbour']):
    label = '{}, {}'.format(neighbor, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Setting up Folium Credentials

In [157]:
CLIENT_ID = 'QGIGC40OIPFLAHA4SZRPLBFFNM1QXA4V2G2LQW5244VYY50P' # your Foursquare ID
CLIENT_SECRET = 'QSE3UMXWQFRONXR4SACUKZJ1HY3HMVYZM1IOT3B1EZXWN0YQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QGIGC40OIPFLAHA4SZRPLBFFNM1QXA4V2G2LQW5244VYY50P
CLIENT_SECRET:QSE3UMXWQFRONXR4SACUKZJ1HY3HMVYZM1IOT3B1EZXWN0YQ


### Exploring the first NeighbourHood


In [161]:
toronto.loc[0]

District          C01
Neighbour    Downtown
Latitude      43.6542
Longitude    -79.3808
Name: 0, dtype: object

In [162]:
neighborhood_latitude = toronto.loc[0, "Latitude"]
neighborhood_longitude = toronto.loc[0, "Longitude"]

#### Lets fetch the top 100 venues within a radius of 500 metres of Downtown

In [164]:
# FourSquare Query URL
LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=QGIGC40OIPFLAHA4SZRPLBFFNM1QXA4V2G2LQW5244VYY50P&client_secret=QSE3UMXWQFRONXR4SACUKZJ1HY3HMVYZM1IOT3B1EZXWN0YQ&v=20180605&ll=43.6541737,-79.3808116451341&radius=500&limit=100'

In [166]:
# GET Request Sent
# type your answer here
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c447bb39fb6b7269aafbb5d'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ad4c062f964a520b5f720e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/performingarts_theater_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d137941735',
         'name': 'Theater',
         'pluralName': 'Theaters',
         'primary': True,
         'shortName': 'Theater'}],
       'id': '4ad4c062f964a520b5f720e3',
       'location': {'address': '189 Yonge St',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'btwn Queen St E & Shuter St',
        'distance': 204,
        'formattedAddress': ['189 Yonge St (btwn Queen St E & Shuter St)',
         'Toronto ON M5B 2H1',
     

In [167]:
#### FUNCTION TO EXTRACT CATEGORY FROM THE VENUE - BORROWED FROM FourSquares Lab
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Explore Neighborhoods in Toronto, Canada

In [168]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [169]:
toronto_venues = getNearbyVenues(names=toronto['Neighbour'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Downtown
Harbourfront
Little Italy
Little Portugal
The Annex
Yorkville
South Hill
Summerhill
Wychwood Park
Deer Park
Forest Hill South
Oakwood–Vaughan
Humewood
Corso Italia
Bedford Park
Lawrence Manor
North Toronto
Forest Hill North
North York
Clanton Park
Bathurst Manor
Willowdale
Newtonbrook West
Westminster
Lansing-Westgate
Cabbagetown
St. Lawrence Market
Toronto waterfront
Moss Park
Church and Wellesley
Garden District
Moore Park
Rosedale
Davisville Village
Lawrence Park South
Leaside
Thorncliffe Park
Flemingdon Park
Don Mills
Parkwoods
Victoria Village
Newtonbrook East
Willowdale East
Hillcrest Village
Bayview Woods – Steeles
Bayview Village
Don Valley Village
Henry Farm
Pleasant View
Riverdale
Danforth
Leslieville
The Beaches
Woodbine - Lumsden
Danforth
East York
Playter
Broadview North (Old East York)
O'Connor–Parkview
Crescent Town
The Golden Mile
Dorset Park
Wexford
Maryvale
Scarborough Junction (Kennedy Park)
Ionview
Clairlea
Steeles
L'Amoreaux
Tam O'Shanter – Sullivan
Birch 

In [172]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown,43.654174,-79.380812,Elgin And Winter Garden Theatres,43.653394,-79.378507,Theater
1,Downtown,43.654174,-79.380812,UNIQLO ユニクロ,43.655910,-79.380641,Clothing Store
2,Downtown,43.654174,-79.380812,Ed Mirvish Theatre,43.655102,-79.379768,Theater
3,Downtown,43.654174,-79.380812,Massey Hall,43.654195,-79.378891,Concert Hall
4,Downtown,43.654174,-79.380812,CF Toronto Eaton Centre,43.654148,-79.380639,Shopping Mall


In [175]:
# Size of the resultant Dataframe
toronto_venues.shape

(2890, 7)

### Venues Returned for Each Neighborhood

In [177]:
toronto_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alderwood,10,10,10,10,10,10
Amesbury,4,4,4,4,4,4
Baby Point,4,4,4,4,4,4
Bathurst Manor,4,4,4,4,4,4
Bayview Village,13,13,13,13,13,13
Bayview Woods – Steeles,2,2,2,2,2,2
Bedford Park,2,2,2,2,2,2
Bendale,6,6,6,6,6,6
Birch Cliff,4,4,4,4,4,4


### Number of Unique Categories of Venues

In [181]:
len(toronto_venues["Venue Category"].unique())

287

### Analysis of each Neighborhood

In [183]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Castle,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Uni

In [185]:
# Resultant Shape of Dataframe
toronto_onehot.shape

(2890, 287)

### Rows grouped by Neighbourhood

In [187]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Castle,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Uni

In [188]:
# Resultant Shape of Dataframe
toronto_grouped.shape

(132, 287)

### Print NeighborHood alonside their top 5 Venues

In [190]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+ hood +"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood----
            venue  freq
0     Pizza Place   0.2
1    Dance Studio   0.1
2        Pharmacy   0.1
3  Sandwich Place   0.1
4     Coffee Shop   0.1


----Amesbury----
                venue  freq
0                Bank  0.25
1                Park  0.25
2  Athletics & Sports  0.25
3         Coffee Shop  0.25
4                 Zoo  0.00


----Baby Point----
         venue  freq
0         Park  0.75
1        River  0.25
2          Zoo  0.00
3  Music Venue  0.00
4       Office  0.00


----Bathurst Manor----
               venue  freq
0     Baseball Field  0.25
1  Convenience Store  0.25
2         Playground  0.25
3               Park  0.25
4                Zoo  0.00


----Bayview Village----
                 venue  freq
0  Sporting Goods Shop  0.08
1          Coffee Shop  0.08
2       Breakfast Spot  0.08
3          Gas Station  0.08
4        Metro Station  0.08


----Bayview Woods – Steeles----
          venue  freq
0          Park   0.5
1       Dog Run   0.5
2           Zoo  